In [2]:
# Imports
import numpy as np
import matplotlib.pylab as plt
import time



---



On considère le problème de l’optimisation d’un portefeuille. On appelle J, la fonction définie sur $\mathbf{R}$ par $J(x) = \frac{1}{2}<x, Ax>_{\mathbf{R}^n}$
On désigne par $K$, l’ensemble des contraintes, soit $K = \{ x \in \mathbf{R}^n \vert <x, u> = 1, ~ <x, e> = r_0 \}$. L’objectif est de résoudre numériquement le problème $\inf_{x \in K} J(x)$. Nous allons ré-écrire notre ensemble $K$ de la façon suivante : $K = \{ x \in Cx = f\}$ avec : 
 $C =$ \begin{pmatrix} 1 & \cdots & 1 \\ e_1 & \cdots & e_n \end{pmatrix}
et $f = (1, r_0)^T$.

A l'aide du théorème des extrémas liés, nous avons les conditions d'optimalités suivante : $A x^* = \lambda_1 u + \lambda_2 e$ où $x^*$ est un minimiseur de $J$ et $\lambda_1$ et $\lambda_2$ sont des multiplicateurs de Lagrange. 

Ainsi on va se fixer deux multiplicateurs de lagrange $\lambda^0_1$ et $\lambda^0_2$ initial et tout d'abord résoudre le problème à l'itération $k$,$\underset{x \in \mathbb{R}^n} {\inf}  L(x, \lambda^k )$  où $\lambda^k$ est connue. On fera cela à l'aide d'un algorithme de gradient à pas constant. Ensuite nous devrons maximiser le problème, $\underset{\lambda \in \mathbb{R}^2}{\sup}  L(x^k, \lambda )$ où $x^k$ est la donnée calculée précédémment. Cela se fera aussi à l'aide d'un algorithme de gradient.



---



On considère une fonction qui permet de générer la matrice $A$. De plus on prend, $r_0 = 2.5$ et $e_i = i$ pour tout $i$ allant de $1$ à $n$.

In [3]:
n = 5
r_0 = 2.5
e = np.arange(1, n+1)
u = np.ones(n)

A = np.diag(e) # matrice diagonale pour l'instant.
R = np.random.rand(n, n) # des nombres entre 0 et 1.
A = A + 0.05 * np.dot(np.transpose(R),R) # produit scalaire entre deux matrices,  
# correspond à un produit de somme sur le dernier axe de la première matrice et l'avant-dernier axe de 
# la deuxième. 

print(A)

[[1.06574804 0.03573978 0.09217373 0.0493276  0.06878074]
 [0.03573978 2.02825208 0.04862192 0.04115198 0.03248549]
 [0.09217373 0.04862192 3.16142946 0.08056469 0.1221193 ]
 [0.0493276  0.04115198 0.08056469 4.09313573 0.07823195]
 [0.06878074 0.03248549 0.1221193  0.07823195 5.1161675 ]]


La dernière ligne permet de rendre notre matrice à diagonale (strictement) dominante. Nous allons le vérifier numériquement.

In [4]:
for i in range(n):
    sum = 0
    for j in range(n):
      if(i != j):
          sum += abs(A[i][j])

    if(abs(A[i][i]) < sum):
        print("Elle n'est pas à diagonale strictement dominante.")
        exit(0)

print("Elle est à diagonale strictement dominante.")

Elle est à diagonale strictement dominante.




---



Nous voulons à présent résoudre le problème d'optimisation à l'aide de la méthode d'Uzawa. Nous aurons donc besoin d'implémenter la fonction $J$ et le lagrangien.

In [5]:
def J(x):
    return 1/2* np.dot(x, A @ x)

def grad_J(x):
    return A @ x

# Nos fonctions de contraintes h1 et h2 :
def h1(x):
    return np.inner(x, u) - 1

def h2(x):
    return np.dot(x, e) - r_0

# Lagrangien et son gradient : 
def L(x, a, b): # a et b sont les multiplicateurs de Lagrange.
    return J(x) + a * h1(x) + b * h2(x)

def grad_L(x, a, b) :
    return grad_J(x) + a * u + b * e

Ensuite on implémente l'algorithme :

In [9]:
# Nos deux pas pour les gradients à pas constants.
rho_1 = 0.001
rho_2 = 0.2

# Fonction pour minimiser L(., a, b) sans contrainte.
def minL(x_last, a, b, rho = rho_2) :

  it = 0
  itmax = 50000

  x = x_last - rho * grad_L(x_last, a, b)
  err = np.linalg.norm(grad_L(x, a, b)) # On veut le gradient nul.

  while (err > 1e-8 and it < itmax):

    x = x - rho * grad_L(x, a, b)
    err = np.linalg.norm(grad_L(x, a, b))
    it +=1 

  return x


def Uzawa(a, b, rho = rho_1):

  # Itération maximale.
  itmax = 50000
  it = 0

  # Minimiser d'abord le problème sans contrainte.
  x = minL(x_0, a, b)

  # Puis maximiser le problème avec contrainte.
  a1 = a + rho * h1(x)
  b1 = b + rho * h2(x)

  # Erreur.
  err = np.linalg.norm(x-x_0) + np.abs(a1-a) + np.abs(b1-b)

  a = a1
  b = b1

  while (err > 1e-8 and it < itmax):
      
    it += 1

    x1 = minL(x, a, b)
    a1 = a + rho * h1(x)
    b1 = b + rho * h2(x)

    err = np.linalg.norm(x - x1) + np.abs(a1 - a) + np.abs(b1 - b)

    x = x1
    a = a1
    b = b1
  return x, it, err

In [10]:
# Il faut initialiser les multiplicateurs de lagrange à l'itération k = 0.
a = 1
b = 1

n = 5

# x_0 initial.
x_0 = np.ones(n)


x_appro, it, err = Uzawa(a, b)

print("La solution est : ", x_appro)
print("Le nombre d'itération obtenue : ", it) 
print("Bonne erreur ? : ", (err < 1e-8))

La solution est :  [0.34180021 0.22413018 0.15698727 0.14640457 0.13067224]
Le nombre d'itération obtenue :  21085
Bonne erreur ? :  True


Un inconvéniant majeur qu'on peut noter pour l'algorithme d'Uzawa est le nombre d'itération très très grand. 